In [6]:
import os
import sys
import gym
import numpy as np
import configparser
from datetime import datetime
from itertools import product
# Import Tensorflow
import tensorflow as tf
import tensorflow.compat.v1 as tf_v1

# Insert module root directory to sys.path

In [1]:
ROOT_DIR = os.path.abspath(os.path.join(os.path.abspath(""), '..'))
sys.path.insert(0, ROOT_DIR)
from run import MountainCar_v0
from src import ReplayBuffer
from src.Utils import get_logger, eval_dict_values

This function generates **init_kwargs** and **train_kwargs**. *init_kwargs* contains keyworded-arguments to pass while initializing the agent whereas *train_kwargs* contains keyworded-arguments to pass while training the agent.

def parameter_generator_old(eps_decays, exp_intervals, target_update_steps, explore_ratios, ddqns, seeds):
    for eps_decay in eps_decays:
        for exp_interval in exp_intervals:
            for target_update_step in target_update_steps:
                for exp_ratio in explore_ratios:
                    for ddqn in ddqns:
                        for seed in seeds:
                            init_kwargs = {"eps_decay": eps_decay, "explore_exploit_interval":exp_interval, "DDQN":ddqn}
                            train_kwargs = {"target_update_steps":target_update_step, "explore_ratio":exp_ratio}
                            yield seed, init_kwargs, train_kwargs

def parameter_generator(eps_decays, exp_intervals, target_update_steps, explore_ratios, ddqns, seeds):
    init_parameters = product(eps_decays, exp_intervals, ddqns)
    train_parameters = product(target_update_steps, explore_ratios)
    for para in product(init_parameters, train_parameters, seeds):
        init_args, train_args, seed = para
        init_kwargs = {"eps_decay": init_args[0], "explore_exploit_interval":init_args[1], "DDQN":init_args[2]}
        train_kwargs = {"target_update_steps":train_args[0], "explore_ratio":train_args[1]}
        yield seed, init_kwargs, train_kwargs

The following function receives parameters like tensorflow **sess**, OpenAI gym **env**, model id and keyworded arguments to initialize and train the agent.  

The function performs following tasks:
- Creates a *ReplayBuffer* to store transitions **(state, action, reward, next_state, done)** of the agent.
- Creates an *agent*
- Creates tf.train.Saver object to save/restore agent model (as checkpoints)
- Trains the agent for given number of episodes, measures goal achievements, logs the summaries of each epoch (average loss, epoch length, maximum position, total epoch reward) in Tensorboard and returns the final summaries
- Saves trained model
- Plots the summaries using matplotlib if **plot_result** is set to True.
- Returns only the goal summary (number of achieved goals, first achieved goal, goal with most reward) where the format of each goal is *(epoch and epoch reward)*

"""
Agent init:
1. Fixed parameters: sess, env, eps, lr, df, tau, render, (change later) [mem, batch_size]
2. Variable parameters: eps_decay, explore_exploit_interval, DDQN

Agent train:
1. Fixed parameters: NUM_EPISODES, display_every, goal_trials, goal_reward
2. Variable parameters: target_update_steps, explore_ratio
"""
def run(sess, env, model_dir, summ_dir, model_i, init_kwargs, train_kwargs, plot_result=False):
    model_name = "Model {}".format(model_i)
    print("# Training: {}".format(model_name))
    summ_dir = os.path.join(summ_dir, model_name)
    # create replay buffer, agent and (model checkpoint) saver
    mem = ReplayBuffer(160_000)
    agent = MountainCar(sess, env, mem, batch_size=100, eps=(1, 0.001), lr=0.97, 
                           df=0.99, tau=1, render=render, summ_dir=summ_dir, 
                           **init_kwargs)
    # TODO: Increase this size
    saver = tf.train.Saver(max_to_keep=100)
    sess.run(tf.global_variables_initializer())
    results= agent.train(NUM_EPISODES, display_every=DISPLAY_RATE,
                         goal_trials=100, goal_reward=-110.0,
                         **train_kwargs)
    saver.save(sess, os.path.join(model_dir, model_name))
    *results, goal_summary = results
    if plot_result:
        for p, plt_name in zip(results, ("Losses", "Rewards", "Max pos", "Epoch length")):
            plt.plot(p)
            plt.xlabel('Episodes')
            plt.ylabel(plt_name)
            plt.show()
            plt.close("all")
    return goal_summary

# Explore logs
**8th Sept 2019 - 22:53** 
- eps_decay:(0.1, 0.5, 3), 
- explore_interval:(5, 20, 4), 
- target_update_steps:(5, 20, 4), 
- explore_ratios:(0.1, 0.5, 3), 
- random seeds:(100, 1000, 2)

**12th Sept 2019 - 23:47** 
- eps_decay:(0.1, 0.5, 5), 
- explore_interval:(10, 40, 4), 
- target_update_steps:(10, 1), 
- explore_ratios:(0.25, 1), 
- random seeds:(100, 1000, 2)

In [ ]:
LOG_DIR = "log"
SUMM_DIR = "summaries"
CONFIG_DIR = "config"
ENV_NAME = "MountainCar-v0"
SEEDS = np.random.randint(100, 1000, size=1, dtype=np.uint16)
TF_CONFIG = tf_v1.ConfigProto(gpu_options=tf_v1.GPUOptions(per_process_gpu_memory_fraction=0.5), 
                              allow_soft_placement=True)

In [ ]:
if __name__ == "__main__":
    # Default summary directoy, log and config file
    date_time = datetime.now().strftime("%d.%m.%Y %H.%M")
    summ_dir = os.path.join(SUMM_DIR, "{} {}".format(ENV_NAME, date_time))
    log_file = os.path.join(summ_dir, LOG_DIR, "Results {} {}.log".format(ENV_NAME, date_time))
    config_file = os.path.join(CONFIG_DIR, "MountainCar-v0.ini")
    # Parse configuration from config file
    config_parser = configparser.ConfigParser()
    """
    Somehow in Jupyter Notebook config_parser keeps looking in ROOT_DIR\notebooks\config_dir 
    instead of ROOT_DIR\config_dir, even though the ROOT_DIR is in sys.path. 
    Therefore, explicit addressing is required in Jupyter Notebook.
    """
    config_parser.read(os.path.join(ROOT_DIR, config_file))
    # Load configurations from config file
    init_kwargs = eval_dict_values(config_parser["init_kwargs"])
    train_kwargs = eval_dict_values(config_parser["train_kwargs"])
    mem_size = eval(config_parser["others"].get("mem_size", 50_000))
    # Setup logger
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logger = get_logger(log_file)
    # Create environment and replay buffer
    env = gym.make(ENV_NAME)
    mem = ReplayBuffer(mem_size)
    if record_interval > 0:
        # Wrap environment with Monitor wrapper to record videos
        env = gym.wrappers.Monitor(env, os.path.join(summ_dir, "videos"), force=True,
                                   video_callable=lambda epoch: not epoch%record_interval)
    # Testing model
    if test_model_chkpt is not None:
        # Override goal_trials and display_every parameters
        train_kwargs["goal_trials"] = 1
        train_kwargs["display_every"] = train_kwargs["epochs"]/10
    for init_kwargs, train_kwargs in 
    # Run the program
    MountainCar_v0.run(env, SEEDS, mem, logger, summ_dir, init_kwargs, train_kwargs, 
                       log_init_kwargs, log_train_kwargs, plot_result, sess_config=TF_CONFIG, 
                       test_model_chkpt=test_model_chkpt)

The agent is trained with different combinations of the following parameters:

In [4]:
DDQNS = [True]
EPS_DECAYS = np.linspace(0.1, 0.5, num=5, dtype=np.float16)
EXPLORE_EXPLOIT_INTERVALS = np.linspace(10, 40, num=4, dtype=np.uint16)
TARGET_UPDATE_STEPS = [10]#np.linspace(5, 20, num=4, dtype=np.uint16)
EXPLORE_RATIOS = [0.25]#np.linspace(0.1, 0.5, num=3, dtype=np.float16)
SEEDS = np.random.randint(100, 1000, size=2, dtype=np.uint16)

In [1]:
def parameter_generator(eps_decays, exp_intervals, target_update_steps, explore_ratios, ddqns):
    init_parameters = product(eps_decays, ddqns)
    train_parameters = product(target_update_steps, exp_intervals, explore_ratios)
    for param in product(init_parameters, train_parameters):
        init_args, train_args= param
        init_kwargs = {"eps_decay": init_args[0], 
                       "ddqn":init_args[1]}
        train_kwargs = {"target_update_steps":train_args[0], 
                        "explore_exploit_interval":init_args[1], 
                        "explore_ratio":train_args[2]}
        yield init_kwargs, train_kwargs

In [7]:
for k1, k2 in parameter_generator(EPS_DECAYS, EXPLORE_EXPLOIT_INTERVALS, TARGET_UPDATE_STEPS, EXPLORE_RATIOS, DDQNS):
    print(k1, k2)

{'eps_decay': 0.1, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.1, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.1, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.1, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.2, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.2, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.2, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.2, 'ddqn': True} {'target_update_steps': 10, 'explore_exploit_interval': True, 'explore_ratio': 0.25}
{'eps_decay': 0.3, 'ddqn': True} {'target_update_steps':

# Explore plan
1. Set parameters as far apart as possible.
2. Determine parameter ranges that produce reasonably good results
3. Set parameters within this ranges or explore new parameter range
4. Repeat step 2  

# Further plan
- Create class to explore hyperparameters. 
- The class includes parameter_generator and run methods.
- It would probably have a context manager option to save and load trained parameters information
- Store and plot achieved goal information

In [3]:
class Hyperparameter_Checker:
    def __init__(self, env, mem, model_dir, summ_dir, log_file, start_index=1, saver_max_size=100, 
                 batch_size=100, display_every=100, epochs=1000, render=False):
        self.env = env
        self.mem = mem
        self.saver_max_size = saver_max_size
        self.display_every = display_every
        self.start_index = start_index
        self.batch_size = batch_size
        self.epochs = epochs
        self.render = render
        self.model_dir = model_dir
        self.summ_dir = summ_dir
        self.log_file = log_file
        self.logger = get_logger(self.log_file)
        
    @staticmethod
    def parameter_generator(eps_decays, exp_intervals, target_update_steps, explore_ratios, ddqns, seeds):
        init_parameters = product(eps_decays, exp_intervals, ddqns)
        train_parameters = product(target_update_steps, explore_ratios)
        for param in product(init_parameters, train_parameters, seeds):
            init_args, train_args, seed = param
            init_kwargs = {"eps_decay": init_args[0], "explore_exploit_interval":init_args[1], "ddqn":init_args[2]}
            train_kwargs = {"target_update_steps":train_args[0], "explore_ratio":train_args[1]}
            yield seed, init_kwargs, train_kwargs
    
    def run(self, sess, model_i, init_kwargs, train_kwargs, seed=None, plot_result=False):
        model_name = "Model {}".format(model_i)
        print("# Training: {}".format(model_name))
        summ_dir = os.path.join(self.summ_dir, model_name)
        agent = MountainCar(sess, self.env, self.mem, batch_size=self.batch_size, eps_limits=(1, 0.001), 
                               lr=0.97, df=0.99, tau=1, render=self.render, summ_dir=summ_dir, 
                               **init_kwargs)
        saver = tf.train.Saver(max_to_keep=self.saver_max_size)
        sess.run(tf.global_variables_initializer())
        results= agent.train(self.epochs, display_every=self.display_every, 
                             goal_trials=100, goal_reward=-110.0, 
                             **train_kwargs)
        saver.save(sess, os.path.join(self.model_dir, model_name))
        *results, goal_summary = results
        if plot_result:
            for p, plt_name in zip(results, ("Losses", "Rewards", "Max pos", "Epoch length")):
                plt.plot(p)
                plt.xlabel('Episodes')
                plt.ylabel(plt_name)
                plt.show()
                plt.close("all")
        parameter_dict = {"seed":seed, **init_kwargs, **train_kwargs}
        agent.log(self.logger, model_i, parameter_dict, goal_summary)
    
    def check_parameters(self, *args, plot_result=False):
        para_gen = self.parameter_generator(*args)
        for model_i, (seed, init_kwargs, train_kwargs) in enumerate(para_gen, start=1):
            if model_i < self.start_index:
                continue                        # Skip parameters until the model_i >= start_index 
            self.env.seed(int(seed))
            with tf.Session(config=TF_CONFIG) as sess:
                self.run(sess, model_i, seed=seed, init_kwargs=init_kwargs,
                         train_kwargs=train_kwargs, plot_result=plot_result)
            tf.compat.v1.reset_default_graph()
            self.mem.clear()
             """
            parameter_str = dict2str(parameter_dict)
            self.logger.debug("Model {:<2} - {}".format(model_i, parameter_str))
            num_goals, first_goal, max_goal = goal_summary
            first_goal_epoch, first_goal_reward = first_goal 
            max_goal_epoch, max_goal_reward = max_goal
            self.logger.debug("Goals achieved: {}".format(num_goals))
            if num_goals:
                self.logger.info("First goal achieved: {:.2f} mean reward at {} epoch.".format(first_goal_reward, first_goal_epoch))
            self.logger.info("Max goal achieved: {:.2f} mean reward at {} epoch.\n".format(max_goal_reward, max_goal_epoch))"""

Global variables

In [4]:
LOG_DIR = "Log"
ENV_NAME = "MountainCar-v0"
DISPLAY_RATE = 250
NUM_EPISODES = 1000
render = False
TF_CONFIG = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.25), 
                           allow_soft_placement=True)
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)

Following tasks are performed in the block below:
- Generate different combinations of parameters
- Initialize the agent with parameters in **init_kwargs**
- Train the agent with parameters in **train_kwargs**
- Reset tensorflow graph (to reuse the graph to train future models)
- Log goal summary

In [5]:
if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    mem = ReplayBuffer(50_000)
    """model_dir = r'models\Model 08.09.2019 23.05'
    summ_dir = r'summaries\MountainCar-v0 08.09.2019 23.05'
    log_file = r'Log\Results 08.09.2019 23.05.txt'"""
    date_time = datetime.now().strftime("%d.%m.%Y %H.%M")
    model_dir = os.path.join("models", "Model {}".format(date_time))
    summ_dir = os.path.join("summaries", "{} {}".format(ENV_NAME, date_time))
    log_file = os.path.join(LOG_DIR, "Results {}.txt".format(date_time))
    checker = Hyperparameter_Checker(env, mem, model_dir, summ_dir, log_file, start_index=1, 
                                     saver_max_size=300, display_every=DISPLAY_RATE, epochs=NUM_EPISODES,
                                     render=render)
    checker.check_parameters(EPS_DECAYS, EXPLORE_EXPLOIT_INTERVALS, TARGET_UPDATE_STEPS, 
                             EXPLORE_RATIOS, DDQNS, SEEDS)

W0913 00:06:05.869809  8472 deprecation.py:506] From c:\users\raj k\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0913 00:06:05.979811  8472 deprecation.py:323] From c:\users\raj k\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Training: Model 1
Goal: Get average reward of -110.00 over 100 consecutive trials!
Epoch: 250, mean_loss: 2.9343, total_reward: -143.0, max_pos: 0.5059,  in 266.3583 secs
Epoch: 500, mean_loss: 0.8678, total_reward: -103.0, max_pos: 0.5135,  in 184.2229 secs
Epoch: 750, mean_loss: 0.1018, total_reward: -200.0, max_pos: -0.0846,  in 159.7277 secs
Epoch: 1000, mean_loss: 0.2040, total_reward: -97.0, max_pos: 0.5052,  in 155.1219 secs


First goal achieved: -109.99 mean reward at 703 epoch.
Max goal achieved: -105.18 mean reward at 890 epoch.



# Training: Model 2
Goal: Get average reward of -110.00 over 100 consecutive trials!
Epoch: 250, mean_loss: 1.5008, total_reward: -174.0, max_pos: 0.5044,  in 213.1094 secs
Epoch: 500, mean_loss: 0.1313, total_reward: -116.0, max_pos: 0.5105,  in 168.3576 secs
Epoch: 750, mean_loss: 0.1695, total_reward: -171.0, max_pos: 0.5053,  in 163.2149 secs
Epoch: 1000, mean_loss: 0.1052, total_reward: -140.0, max_pos: 0.5369,  in 158.5637 secs


First goal achieved: -109.88 mean reward at 486 epoch.
Max goal achieved: -105.55 mean reward at 556 epoch.



...

First goal achieved: -109.82 mean reward at 516 epoch.
Max goal achieved: -102.39 mean reward at 648 epoch.



# Training: Model 39
Goal: Get average reward of -110.00 over 100 consecutive trials!
Epoch: 250, mean_loss: 2.6428, total_reward: -151.0, max_pos: 0.5369,  in 244.2484 secs
Epoch: 500, mean_loss: 0.3937, total_reward: -91.0, max_pos: 0.5164,  in 185.7556 secs
Epoch: 750, mean_loss: 0.5960, total_reward: -104.0, max_pos: 0.5117,  in 191.7711 secs
Epoch: 1000, mean_loss: 0.6848, total_reward: -108.0, max_pos: 0.5162,  in 165.2275 secs


First goal achieved: -109.89 mean reward at 883 epoch.
Max goal achieved: -106.78 mean reward at 918 epoch.



# Training: Model 40
Goal: Get average reward of -110.00 over 100 consecutive trials!
Epoch: 250, mean_loss: 1.0653, total_reward: -197.0, max_pos: 0.5369,  in 246.8239 secs
Epoch: 500, mean_loss: 0.2191, total_reward: -126.0, max_pos: 0.5440,  in 180.1326 secs
Epoch: 750, mean_loss: 0.1431, total_reward: -153.0, max_pos: 0.5086,  in 183.2748 secs
Epoch: 1000, mean_loss: 0.3423, total_reward: -142.0, max_pos: 0.5015,  in 202.7662 secs


First goal achieved: -109.53 mean reward at 442 epoch.
Max goal achieved: -102.61 mean reward at 486 epoch.



if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    date_time = datetime.now().strftime("%d.%m.%Y %H.%M")
    model_dir = os.path.join("models", "Model {}".format(date_time))
    summ_dir = os.path.join("summaries", "{} {}".format(ENV_NAME, date_time))
    log_file = os.path.join(LOG_DIR, "Results {}.txt".format(date_time))
    logger = get_logger(log_file)
    para_gen = parameter_generator(EPS_DECAYS, EXPLORE_EXPLOIT_INTERVALS, TARGET_UPDATE_STEPS, 
                                   EXPLORE_RATIOS, DDQNS, SEEDS)
    for model_i, (seed, init_kwargs, train_kwargs) in enumerate(para_gen, start=1):
        env.seed(int(seed))
        with tf.Session(config=TF_CONFIG) as sess:
            goal_summary = run(sess, env, model_dir, summ_dir, model_i, init_kwargs=init_kwargs,
                               train_kwargs=train_kwargs, plot_result=False)
        tf.compat.v1.reset_default_graph()
        # Log results
        parameter_dict = {"seed":seed, **init_kwargs, **train_kwargs} 
        parameter_str = dict2str(parameter_dict)
        logger.debug("Model {:<2} - {}".format(model_i, parameter_str))
        num_goals, first_goal, max_goal = goal_summary
        first_goal_epoch, first_goal_reward = first_goal 
        max_goal_epoch, max_goal_reward = max_goal
        logger.debug("Goals achieved: {}".format(num_goals))
        if num_goals:
            logger.info("First goal achieved: {:.2f} mean reward at {} epoch.".format(first_goal_reward, first_goal_epoch))
        logger.info("Max goal achieved: {:.2f} mean reward at {} epoch.\n".format(max_goal_reward, max_goal_epoch))